# Introduction to `fastplotlib`

This notebook goes the basic components of the `fastplotlib` API, image, image updates, line plots, and scatter plots. 

# Documentation

The docs are here: http://fastplotlib.readthedocs.io/

The link is also on the repo, in general you can always find the docs for a package by looking for the link in the repo's README.

# Using notebooks

## Modify and make notes in these demo notebooks throughout the workshop, and then push them to your fork. You'll then have a snapshot of everything you learned at the workshop!

In [ ]:
from fastplotlib import Plot
from ipywidgets import VBox, HBox, IntSlider
import numpy as np

### Simple image

In [ ]:
# create a `Plot` instance
plot = Plot()

# make some random 2D image data
data = np.random.rand(512, 512)

# plot the image data
image_graphic = plot.add_image(data=data, name="random-image")

# show the plot
plot.show()

### Use the handle on the bottom right corner of the _canvas_ to resize it. You can also pan and zoom using your mouse!

Changing graphic "features"

In [ ]:
image_graphic.cmap = "viridis"

In [ ]:
image_graphic.data = 0

In [ ]:
image_graphic.data[::15, :] = 1
image_graphic.data[:, ::15] = 1

In [ ]:
image_graphic.data = np.random.rand(512, 512)

### Plots are indexable and give you their graphics by name

In [ ]:
plot.graphics

In [ ]:
plot["random-image"]

#### The `Graphic` instance is also returned when you call `plot.add_<graphic_type>`.

In [ ]:
image_graphic

In [ ]:
image_graphic is plot["random-image"]

## If you don't have a powerful GPU close the canvas to free it up!

If you are confident that you have a powerful GPU you can leave this open.

In the next release we will make this simpler by allowing `plot.close()`, but for now you have to use `plot.canvas.close()`

In [ ]:
plot.canvas.close()

### Image updates

This examples show how you can define animation functions that run on every render cycle.

In [ ]:
# create another `Plot` instance
plot_v = Plot()

plot.canvas.max_buffered_frames = 1

# make some random data again
data = np.random.rand(512, 512)

# plot the data
plot_v.add_image(data=data, name="random-image")

# a function to update the image_graphic
# a plot will pass its plot instance to the animation function as an arugment
def update_data(plot_instance):
    new_data = np.random.rand(512, 512)
    plot_instance["random-image"].data = new_data

#add this as an animation function
plot_v.add_animations(update_data)

# show the plot
plot_v.show()

### We can share controllers across plots

This example creates a new plot, but it synchronizes the pan-zoom controller

In [ ]:
plot_sync = Plot(controller=plot_v.controller)

data = np.random.rand(512, 512)

image_graphic_instance = plot_sync.add_image(data=data, cmap="viridis")

# you will need to define a new animation function for this graphic
def update_data_2():
    new_data = np.random.rand(512, 512)
    # alternatively, you can use the stored reference to the graphic as well instead of indexing the Plot
    image_graphic_instance.data = new_data

plot_sync.add_animations(update_data_2)

plot_sync.show()

#### Keeping a reference to the Graphic instance, as shown above `image_graphic_instance`, is useful if you're creating something where you need flexibility in the naming of the graphics

### You can also use `ipywidgets.VBox` and `HBox` to stack plots. See the `gridplot` notebooks for a proper gridplot interface for more automated subplotting

In [ ]:
VBox([plot_v.show(), plot_sync.show()])

# Close the canvases

In [ ]:
plot_v.canvas.close()
plot_sync.canvas.close()

# Line plots

## 2D line plots

This example plots a sine wave, cosine wave, and ricker wavelet and demonstrates how **Graphic Features** can be modified by slicing!

### First generate some data.

In [ ]:
# linspace, create 100 evenly spaced x values from -10 to 10
xs = np.linspace(-10, 10, 100)
# sine wave
ys = np.sin(xs)
sine = np.dstack([xs, ys])[0]

# cosine wave
ys = np.cos(xs) + 5
cosine = np.dstack([xs, ys])[0]

# sinc function
a = 0.5
ys = np.sinc(xs) * 3 + 8
sinc = np.dstack([xs, ys])[0]

### We will plot all of it on the same plot. Each line plot will be an individual Graphic, you can have any combination of graphics on a plot.

In [ ]:
# Create a plot instance
plot_l = Plot()

# plot sine wave, use a single color
sine_graphic = plot_l.add_line(data=sine, thickness=5, colors="magenta", name="sine")

plot_l.show()

Add some more lines

In [ ]:
# you can also use colormaps for lines!
cosine_graphic = plot_l.add_line(data=cosine, thickness=12, cmap="autumn")

# or a list of colors for each datapoint
colors = ["r"] * 25 + ["purple"] * 25 + ["y"] * 25 + ["b"] * 25
sinc_graphic = plot_l.add_line(data=sinc, thickness=5, colors = colors)

### We can see that there is a cosine `LineGraphic`, but not the sinc Graphic. Use `Plot.auto_scale`

`auto_scale()` is called automatically when you use `show()`, however if you add/modify the `scene` within the plot after calling `show()` you may have to call `auto_scale()`.

In [ ]:
plot_l.auto_scale()

### This plot has 3 graphics

Only one of them is named, it is not required to name graphics but it can be useful for organization.

In [ ]:
plot_l.graphics

## Graphic features support slicing! :D 

In [ ]:
# indexing colors
cosine_graphic.colors[:15] = "green"

In [ ]:
cosine_graphic.colors[80:95] = "magenta"
cosine_graphic.colors[60] = "w"

In [ ]:
# indexing to assign colormaps to entire lines or segments
sinc_graphic.cmap[10:50] = "gray"

In [ ]:
# just like with images if the graphic is named you can use the name
plot_l["sine"].cmap = "seismic"

In [ ]:
# let's take a snapshot so we can compare before and after the next command
plot_l.canvas.snapshot()

In [ ]:
# more complex indexing, set the green value directly from an array
# remember, [n_points, RGBA]
green_vals = np.linspace(0, 1, num=90-65)
cosine_graphic.colors[65:90, 1] = green_vals

In [ ]:
# green vals is just evenly spaced values between 0-1
green_vals

In [ ]:
# more complex indexing of colors
# from point 15 - 30, set every 3rd point as "cyan"
cosine_graphic.colors[15:50:3] = "cyan"

## Graphic _data_ is also indexable

In [ ]:
cosine_graphic.data[10:50:5, :2] = sine[10:50:5]
cosine_graphic.data[90:, 1] = 7

In [ ]:
cosine_graphic.data[0] = np.array([[-10, 0, 0]])

### Toggle the presence of a graphic within the scene

In [ ]:
sinc_graphic.present = False

### scale the plot depending on graphics present in the scene

In [ ]:
plot_l.auto_scale()

In [ ]:
sinc_graphic.present = True

In [ ]:
plot_l.auto_scale()

# Close the canvas

In [ ]:
plot_l.canvas.close()

### 3D line plot

In [ ]:
# just set the camera as "3d", the rest is basically the same :D 
plot_l3d = Plot(camera='3d')

# create a spiral
phi = np.linspace(0, 30, 200)

xs = phi * np.cos(phi)
ys = phi * np.sin(phi)
zs = phi

# use 3D data
# note: you usually mix 3D and 2D graphics on the same plot
spiral = np.dstack([xs, ys, zs])[0]

plot_l3d.add_line(data=spiral, thickness=2, cmap='winter')

plot_l3d.show()

In [ ]:
# close canvas
plot_l3d.canvas.close()

# Scatter plots

### Plot tens of thousands or millions of points

#### There might be a small delay for a few seconds before the plot shows, this is due to shaders being compiled and a few other things. The plot should be very fast and responsive once it is displayed and future modifications should also be fast!

In [ ]:
from fastplotlib import Plot
from ipywidgets import VBox, HBox, IntSlider
import numpy as np

In [ ]:
# create a random distribution of 10,000 xyz coordinates
n_points = 10_000

# if you have a good GPU go for 1.5 million points by uncommenting this :D 
n_points = 500_000

# dimensions always have to be [n_points, xyz]
dims = (n_points, 3)

clouds_offset = 15

# create some random clouds
normal = np.random.normal(size=dims, scale=5)
# stack the data into a single array
cloud = np.vstack(
    [
        normal - clouds_offset,
        normal,
        normal + clouds_offset,
    ]
)

# color each of them separately
colors = ["yellow"] * n_points + ["cyan"] * n_points + ["magenta"] * n_points

# create plot
plot_s = Plot()

# use an alpha value since this will be a lot of points
scatter_graphic = plot_s.add_scatter(data=cloud, sizes=4, colors=colors, alpha=0.8)

plot_s.show()

### scatter graphic features work similarly to line graphic

In [ ]:
# half of the first cloud's points to red
scatter_graphic.colors[:n_points:2] = "r"

In [ ]:
# set the green value directly
scatter_graphic.colors[n_points:n_points * 2, 1] = 0.3

In [ ]:
# set color values directly using an array
scatter_graphic.colors[n_points * 2:] = np.repeat([[1, 1, 0, 0.5]], n_points, axis=0)

In [ ]:
# change the data, change y-values
scatter_graphic.data[n_points:n_points * 2, 1] += 15

In [ ]:
# set x values directly but using an array
scatter_graphic.data[n_points:n_points * 2, 0] = np.linspace(-40, 0, n_points)

# Kill the kernel before moving on to the next demo!